In [1]:
%pip install --upgrade pip
%pip install transformers datasets torch sentencepiece jupyter accelerate



Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from transformers import AutoTokenizer

print("Torch version:", torch.__version__)

model_name = "ai-forever/mGPT"  
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Tokenizer loaded successfully!")


Torch version: 2.6.0+cpu
Tokenizer loaded successfully!


In [3]:
dataset_path = r"C:\Users\Dell\Desktop\Bollywood\cleaned_songs_dataset.txt"

with open(dataset_path, "r", encoding="utf-8") as f:
    songs = f.read().split("\n=== ===\n")  

print(f"Total Songs in Dataset: {len(songs)}")
print("\nSample Song:\n", songs[0][:200])  # Print first 200 characters of the first song


Total Songs in Dataset: 72

Sample Song:
 आज दिल शायराना शायराना शायराना 
शायराना शायराना लगता है 
आज दिल शायराना शायराना शायराना 
शायराना शायराना लगता है 
उड़ता फिर दिल, उतरना जाने ना 
आज दिल शायराना शायराना शायराना 
शायराना शायराना लगता है 



In [4]:
def tokenize_function(texts):
    return tokenizer(texts, truncation=True, padding="max_length", max_length=256)

tokenized_songs = [tokenize_function(song) for song in songs]

print("Sample Tokenized Song (First 10 Tokens):", tokenized_songs[0]["input_ids"][:10])


Sample Tokenized Song (First 10 Tokens): [1586, 6892, 636, 36054, 636, 1224, 636, 6892, 636, 36054]


In [5]:
# Convert tokenized songs into a dataset with input_ids and labels
from datasets import Dataset
train_dataset = Dataset.from_dict({
    "input_ids": [song["input_ids"] for song in tokenized_songs],
    "labels": [song["input_ids"] for song in tokenized_songs]  # Labels should be same as input_ids for causal LM
})

# Print dataset details
print(train_dataset)
print("\nSample Encoded Song:", train_dataset[0]["input_ids"][:20])  # Print first 20 tokens


Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 72
})

Sample Encoded Song: [1586, 6892, 636, 36054, 636, 1224, 636, 6892, 636, 36054, 636, 1224, 636, 227, 205, 3977, 636, 36054, 636, 1224]


In [6]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model_name = "ai-forever/mGPT"
model = AutoModelForCausalLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./hindi_song_model",
    evaluation_strategy="no",  
    save_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir="./logs",
    logging_steps=500,
    num_train_epochs=5,  
    save_total_limit=2,
    report_to="none"
)


print("Model loaded successfully!")


Model loaded successfully!


c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
